# Industry Accelerators - Financial Markets Customer Offer Affinity Prediction Model

### Introduction

Now that we have built the machine learning models, stored and deployed them using [ibm-watson-machine-learning](http://ibm-wml-api-pyclient.mybluemix.net) , we can use the models to score new data. 


Before executing this notebook on IBM Cloud , you need to:<br>
1) When you import this project on an IBM Cloud environment, a project access token should be inserted at the top of this notebook as a code cell. <br>
If you do not see the cell above, Insert a project token: Click on **More -> Insert project token** in the top-right menu section and run the cell <br>

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)
2) Provide your IBM Cloud API key in the subsequent cell<br>
3) You can then step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.<br>


#### Insert IBM Cloud API key
Your Cloud API key can be generated by going to the [API Keys section of the Cloud console](https://cloud.ibm.com/iam/apikeys). From that page, scroll down to the API Keys section, and click Create an IBM Cloud API key. Give your key a name and click Create, then copy the created key and paste it below. 

If you are running this notebook on cloud pak for data on-prem, leave the ibmcloud_api_key field blank.

In [2]:
ibmcloud_api_key = ''

In [3]:
try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project() 

# Create and Test Scoring Pipeline


In this notebook we will:

* Programmatically get the ID's for the deployment space and model deployments that were created in the 2-model-training notebook.
* Promote assets required for scoring new data into the deployment space.
* Create a deployable function which will take raw data for scoring, prep it into the format required for the models and score it.
* Deploy the function.
* Create the required payload, invoke the deployed function and return predictions.



In [4]:

import os
import pandas as pd
import datetime
import json
from ibm_watson_machine_learning import APIClient
import os

if ibmcloud_api_key != '':
    wml_credentials = {
        "apikey": ibmcloud_api_key,
        "url": 'https://' + os.environ['RUNTIME_ENV_REGION'] + '.ml.cloud.ibm.com'
    }
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5"
     }
client = APIClient(wml_credentials)

### User Inputs

Enter the path to the csv file with raw data to be scored and a list of products to be predicted for. The file will be downloaded to the local path to promote it to deployment space later.

In [5]:
# specify the location of the csv file with raw data that we would like to score for

filename='customerProductHistory.csv'
product_ids = ['EDUCATION', 'CASH', 'BROKERAGE', 'FINANCIALPLAN', 'RETIREMENTPLAN']

In [6]:
f = open(filename, 'w+b')
f.write(project.get_file(filename).getbuffer())
f.close()

### Set up Deployment Space, Deployments and Assets

The following code programmatically gets the deployment space and the model deployment details which were created in **2-model-training**. 
We use the space name and deployment names that were used when creating the deployments as specified below. 
If multiple deployments within the selected space have the same name, the most recently created deployment is used. 

Alternatively, the user can manually enter the space and deployment id's.

The code also promotes some assets into the deployment space, specifically, the dataset with raw data for scoring, the python script file which is used for prepping the data and the metadata that was stored when prepping the data. By promoting these assets into the deployment space, they are available and can be accessed by the deployed function. 

In [7]:
space_name = 'Customer Offer Affinity Space'

# loop through each product name to get the model and deployment tags
dict_model_tag = {}
dict_deployment_tag = {}
for product_id in product_ids: 
    dict_model_tag[product_id] = 'offer_affinity_' + product_id + '_model'
    dict_deployment_tag[product_id] = 'offer_affinity_' + product_id + '_model_deployment'

Get the space we are working in, which is found using the name that was specified in **2-model-training**. <br>
If the user would like to use a different space manually set the **space_id**.

Set the space as the default space for working.

In [8]:
l_space_details = []
l_space_details_created_times = []
for space_details in client.spaces.get_details()['resources']:
    if space_details['entity']['name'] == space_name:
        space_id=space_details['metadata']['id']

# set this space as default space
client.set.default_space(space_id)

'SUCCESS'

Get the deployment ids of all the models deployed in **2-model-training** . If there are multiple deployments with the same name in the same space, we take the latest.

In [9]:
model_deployments_dict = {}
for product_id, deployment_name in dict_deployment_tag.items():
    # get the id of the deployments - again, found using the tags that were hardcoded
    # if there are multiple deployments with the same tag in the same space, we take the latest
    l_deployment_details = []
    l_deployment_details_created_times = []
                
    for deployment in client.deployments.get_details()['resources']:
        if deployment['entity']['name'] == deployment_name:            
                l_deployment_details.append(deployment)
                l_deployment_details_created_times.append(datetime.datetime.strptime(deployment['metadata']['created_at'],  '%Y-%m-%dT%H:%M:%S.%fZ'))

    # get the index of the latest created date from the list and use that to get the deployment_id
    list_latest_index = l_deployment_details_created_times.index(max(l_deployment_details_created_times))
    deployment_id = l_deployment_details[list_latest_index]['metadata']['id']
    model_deployments_dict[product_id] = deployment_id

Promote the assets into the deployment space. We will use the prep script for getting the raw data into the format required for scoring. We also need the prep metadata that was saved as json during the prep for training, this ensures that the user inputs specified for prepping the data for training are the same used for scoring. We add these assets into the deployment space. Also store the raw data dataset in the deployment space.

In [10]:
asset_details_json = client.data_assets.create('training_user_inputs_and_prepped_column_names_and_means.json', file_path='training_user_inputs_and_prepped_column_names_and_means.json')
asset_details_script = client.data_assets.create('offer_affinity_prep.py', file_path='offer_affinity_prep.py')
dataset_asset_details = client.data_assets.create(filename, file_path=filename)


Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS


### Create the Deployable Function
Functions can be deployed in Watson Machine Learning in the same way models can be deployed. The python client or REST API can be used to send data to the deployed function. Using the deployed function allows us to prepare the data and pass it to the model for scoring all within the deployed function.

We start off by creating the dictionary of default parameters to be passed to the function. We get the ID's of all assets that have been promoted into the deployment space. We also add the model deployment ID and space ID information into the dictionary.

In [14]:
# get the assets that were stored in the space - in this version of the package we need to manually assign the id
metadata_id = asset_details_json['metadata']['guid']
prep_id = asset_details_script['metadata']['guid']
dataset_id = dataset_asset_details['metadata']['guid']

In [15]:
params = {'space_id' : space_id}
assets_dict = {'dataset_asset_id' : dataset_id, 'metadata_asset_id' : metadata_id, 
                   'prep_script_asset_id' : prep_id, 'dataset_name' : filename}

In [16]:
# create the wml_credentials again. After already creating the client using the credentials, the instance_id gets updated to 999
# re-create the dictionary so that the correct instance_id is used

#wml_credentials["instance_id"] = "openshift"
ai_parms = {'wml_credentials' : wml_credentials,'space_id' : space_id, 'assets' : assets_dict, 'model_deployment_id' : model_deployments_dict}

#### Scoring Pipeline Function

The function below takes new customers to be scored as a payload. It preps the customer raw data, loads the models, executes the model scoring and generates the predictions for each product. 

The following rules are required to make a valid deployable function:

* The deployable function must include a nested function named "score".
* The score function accepts a list.
* The list must include an array with the name "values".
* The score function must return an array with the name "predictions", with a list as the value, which in turn contains an array with the name "values". Example: ```{"predictions" : [{'values' : }]}```
* We pass default parameters into the function, credentials and space detail, details of the assets that were promoted into the space and also the model deployment guid. 
* The assets are downloaded into the deployment space and imported as variables. The raw data to be scored is then prepared and the function calls the model deployment endpoint to score and return predictions. 

In [17]:
def scoring_pipeline(parms=ai_parms):
     
    import pandas as pd
    import requests
    import os
    import json
    
    from ibm_watson_machine_learning import APIClient
    client = APIClient(parms["wml_credentials"])
    client.set.default_space(parms['space_id'])
    

    # call the function to download the stored dataset asset and return the path
    dataset_path = client.data_assets.download(parms['assets']['dataset_asset_id'], parms['assets']['dataset_name'])
    df_raw = pd.read_csv(dataset_path, infer_datetime_format=True, 
                             parse_dates=['CUSTOMER_EFFECTIVE_DATE', 'CUSTOMER_RELATIONSHIP_START_DATE', 
                                              'CUSTOMER_SUMMARY_END_DATE'])
    


    # call the function to download the prep script and return the path
    prep_script_path = client.data_assets.download(parms['assets']['prep_script_asset_id'], 'prep_data_script.py')
    # remove the rest of path and .py at end of file name to get the name of the script for importing
    script_name = os.path.basename(prep_script_path).replace('.py', '')
    
    
            # call the function to download the prep metadata and return the path
    metadata_path = client.data_assets.download(parms['assets']['metadata_asset_id'], 'user_inputs.json')
    def prep(cust_id, scoring_date):
        import requests
        import os
        # import the prep script that we downloaded into the deployment space
        prep_data_script = __import__(script_name)
    

        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        globals().update(metadata)
        
        input_df = df_raw[df_raw[customer_id_col] == cust_id]
        
        scoring_prep = prep_data_script.OfferAffinityPrep('score', product_id=product_list, effective_date_earliest=effective_date_earliest,
                                            effective_date_latest=effective_date_latest, nulls_threshold=nulls_threshold,
                                            max_num_cat_cardinality=max_num_cat_cardinality, customer_id_col=customer_id_col,
                                            customer_effective_date_col=customer_effective_date_col, customer_relationship_start_date_col=customer_relationship_start_date_col,
                                            customer_summary_end_date=customer_summary_end_date, customer_product_summary_end_date_col=customer_product_summary_end_date_col,
                                            required_product_attributes=required_product_attributes, default_attributes=default_attributes, scoring_date=scoring_date)
        
        prepped_data_dict = scoring_prep.prep_data(input_df, 'score')
        
        for product_id in product_list:
            if prepped_data_dict[product_id] is None:
                print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
                return None

            # handle empty data
            if prepped_data_dict[product_id].shape[0] == 0:
                print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
                return None
        
            # if a column does not exist in scoring but is in training, add the column to scoring dataset
            for col in cols_used_for_training[product_id]:
                if col not in list(prepped_data_dict[product_id].columns):
                    prepped_data_dict[product_id][col] = 0

            # if a column exists in scoring but not in training, delete it from scoring dataset
            for col in list(prepped_data_dict[product_id].columns):
                if col not in cols_used_for_training[product_id]:
                    prepped_data_dict[product_id].drop(col, axis=1, inplace=True)

            # make sure order of scoring columns is same as training dataset
            prepped_data_dict[product_id] = prepped_data_dict[product_id][cols_used_for_training[product_id]]
        
            # fill in any missing data - in our metadata we had a key called 'col_means' which was converted into
            # a variable called 'col_means' in the 'globals().update(metadata)' line of code above
            # this new variable is a dictionary with a key for each product 
            # in turn, the value associated with each of these keys is a dictionary with key for each column 
            # and value being the mean value for that column from the data used for training 
            for col, col_mean_value in col_means[product_id].items():
                # only update means if the product is in the list of training columns
                if col in cols_used_for_training[product_id]:
                    prepped_data_dict[product_id][col].fillna(col_mean_value, inplace=True)
        
            # if logistic regression is used as final model for any product, the last cell in 2-model-training appended
            # variable scaling data into the metadata json file
            # if any of the keys created in that step, cols_to_standardise, scaler_means and scaler_standard_dev, are present in the 
            # json file we need to standardise our data
            if 'scaler_means' in metadata:
                # if the above is True, next check if logistic regression was used for this product, ie is the product listed as a key in the nested dictionary
                # only if the product is listed as a key does it mean logistic regression was used
                # using globals().update created a new variable for each of cols_to_standardise, scaler_means and scaler_standard_dev
                if product_id in scaler_means:
                    # loop through each column to be standardised for the product, get the corresponding mean and standard deviation value calculated from training data
                    for i in range(0, len(cols_to_standardise[product_id])):
                        current_col = cols_to_standardise[product_id][i]
                        current_col_mean = scaler_means[product_id][i]
                        current_col_standard_dev = scaler_standard_dev[product_id][i]
                        # scale the variable 
                        prepped_data_dict[product_id][current_col] = (prepped_data_dict[product_id][current_col] - current_col_mean) / current_col_standard_dev
                                 
        return prepped_data_dict
    
    def score(payload):
        import json
        
        scoring_date = payload['input_data'][0]['values']
        cust_id = payload['input_data'][0]['cust_id']
        
        prepped_data_dict = prep(cust_id, scoring_date)
        
        result = {}
        for product_id, prepped_data in prepped_data_dict.items():
            # handle empty data
            if prepped_data is None:
                return {"predictions" : [{'values' : 'Data prep filtered out customer data. Unable to score.'}]}
            elif prepped_data.shape[0] == 0:
                return {"predictions" : [{'values' : 'Data prep filtered out customer data. Unable to score.'}]}
            else:
                #scoring_url = parms['wml_credentials']["url"] + "/v4/deployments/" + parms['model_deployment_id'][product_id] + "/predictions"
                scoring_payload = {"input_data":  [{ "values" : prepped_data.values.tolist()}]}
                
                response_scoring = client.deployments.score(parms['model_deployment_id'][product_id], scoring_payload)
                result[product_id] = response_scoring
                
        
        return {"predictions" : [{'values' : result}]}
    
    return score

### Deploy the Function

The user can specify the name of the function and deployment in the code below. As we have previously seen, we use tags in the metadata to allow us to programmatically identify the deployed function. 

In [18]:
# store the function and deploy it 
function_name = 'offer_affinity_scoring_pipeline_function'
function_deployment_name = 'offer_affinity_scoring_pipeline_function_deployment'


The Software Specification refers to the runtime used in the Notebook, WML training and WML deployment. We use the software specification `default_py3.7` to store the function. We get the ID of the software specification and include it in the metadata when storing the function. Available Software specifications can be retrieved using `client.software_specifications.list()`.



In [19]:
software_spec_id = client.software_specifications.get_uid_by_name("default_py3.7")

In [20]:
# add the metadata for the function and deployment    
meta_data = {
    client.repository.FunctionMetaNames.NAME : function_name,
    client.repository.FunctionMetaNames.TAGS : [ 'offer_affinity_scoring_pipeline_function_tag'],
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id
}

function_details = client.repository.store_function(meta_props=meta_data, function=scoring_pipeline)

function_id = function_details["metadata"]["id"]

meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: function_deployment_name,
    client.deployments.ConfigurationMetaNames.TAGS : ['offer_affinity_scoring_pipeline_function_deployment_tag'],
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the stored model
function_deployment_details = client.deployments.create(artifact_uid=function_id, meta_props=meta_props)



#######################################################################################

Synchronous deployment creation for uid: '240c1b28-309b-4f9e-8b57-cdf4b58e6c5a' started

#######################################################################################


initializing.....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2f6fbe90-87ca-4e7a-bfc9-2217750b5ee3'
------------------------------------------------------------------------------------------------




### Score New Data

Get the guid of the deployed function, create the payload and use the python client to score the data. The deployed function returns the classification prediction along with the probabilities. 

The payload cantains two values. The first is the effective date for scoring. This is the date that the prediction is computed. The second value contains the ID of the customer who we would like to make the prediction for. 

In [22]:
scoring_deployment_id = client.deployments.get_uid(function_deployment_details)
client.deployments.get_details(scoring_deployment_id)

{'entity': {'asset': {'id': '240c1b28-309b-4f9e-8b57-cdf4b58e6c5a'},
  'custom': {},
  'deployed_asset_type': 'function',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XS', 'num_nodes': 1},
  'name': 'offer_affinity_scoring_pipeline_function_deployment',
  'online': {},
  'space_id': '9883d465-8db2-4698-a89b-1871fd894937',
  'status': {'online_url': {'url': 'https://eu-de.ml.cloud.ibm.com/ml/v4/deployments/2f6fbe90-87ca-4e7a-bfc9-2217750b5ee3/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-03-24T04:19:37.770Z',
  'id': '2f6fbe90-87ca-4e7a-bfc9-2217750b5ee3',
  'modified_at': '2021-03-24T04:19:37.770Z',
  'name': 'offer_affinity_scoring_pipeline_function_deployment',
  'owner': 'IBMid-550003B08R',
  'space_id': '9883d465-8db2-4698-a89b-1871fd894937',
  'tags': ['offer_affinity_scoring_pipeline_function_deployment_tag']}}

In [23]:
cust_id = 1000

payload = [{'values' : "2018-09-30", 'cust_id' : cust_id}]

payload_metadata = {client.deployments.ScoringMetaNames.INPUT_DATA: payload}
# score
funct_output = client.deployments.score(scoring_deployment_id, payload_metadata)
funct_output

{'predictions': [{'values': {'EDUCATION': {'predictions': [{'fields': ['prediction',
        'probability'],
       'values': [[0, [0.7711069131590804, 0.22889308684091958]]]}]},
    'CASH': {'predictions': [{'fields': ['prediction', 'probability'],
       'values': [[1, [0.22989065431647085, 0.770109345683529]]]}]},
    'BROKERAGE': {'predictions': [{'fields': ['prediction', 'probability'],
       'values': [[1, [0.11260159947492482, 0.887398400525075]]]}]},
    'FINANCIALPLAN': {'predictions': [{'fields': ['prediction', 'probability'],
       'values': [[1, [0.281159234491286, 0.718840765508714]]]}]},
    'RETIREMENTPLAN': {'predictions': [{'fields': ['prediction',
        'probability'],
       'values': [[1, [0.2582266109148955, 0.7417733890851045]]]}]}}}]}

**The R Shiny Dashboard invokes this scoring pipeline for visualizing the results.**<br>
**Follow the instructions from Readme to launch R-Shiny dashboard.**


<hr>

Sample Materials, provided under <a href="https://github.com/IBM/Industry-Accelerators/blob/master/CPD%20SaaS/LICENSE" target="_blank" rel="noopener noreferrer">license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2021. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>